In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def findMean(data):
    rows = data.shape[0]
    cols = data.shape[1]
    dataTobeProceessed = data.copy()
    # transpose data 
    transposedData = dataTobeProceessed.transpose()
    finalSumList = []
    countlst =[]
    # finding data without 0 Value
    for i in range (len(transposedData)):
        total= 0
        count = 0
        for j in range (len(transposedData[0])):
            if transposedData[i][j]!=0:
                total= total+ transposedData[i][j]
                count = count + 1
        finalSumList.append(float(total))
        if(count==0):
            count = 1
        countlst.append(float(count))    
    meanVal = [i / j for i, j in zip(finalSumList, countlst)] 
    inds = np.where(data == 0) 
    final_data = data.copy()
    final_data[inds] = np.take(meanVal, inds[1])
    
    final =  np.array(final_data).tolist()
    return final

class NeuralNetwork:
    def __init__(self,x,y,learnrate,epoch):
        self.input      = x                 
        self.y          = y
        self.learnrate  = learnrate
        self.epoch      = epoch
        self.weights1   = np.random.rand(self.input.shape[1],6) 
        self.weights2   = np.random.rand(6,1)
        self.outcome    = np.zeros(self.y.shape)
        self.costDataList   = []

    # Calculating sigmoid
    @staticmethod    
    def sigmoidVal(x):
        return 1.0 / (1.0 + np.exp(-x))
    
    # Calculating Derivative of Sigmoid x(1-x)
    @staticmethod    
    def sigmoidDerivative(x): return x * (1 - x)      
    
    # Calculating cost 
    @staticmethod
    def findVals(targetData,outputData):
        return 0.5*np.sum(np.square(np.subtract(targetData,outputData)))
    
    # Apply Sigmoid on Layer and output
    def forwardMovement(self):
        self.HiddenLayer = self.sigmoidVal(np.dot(self.input, self.weights1))
        self.outcome = self.sigmoidVal(np.dot(self.HiddenLayer, self.weights2))

    def backpropogation(self):
        # derivative of the cost function 
        WeightVal2 = np.dot(self.HiddenLayer.T, ((self.y - self.outcome) * self.sigmoidDerivative(self.outcome)))
        WeightVal1 = np.dot(self.input.T,  (np.dot((self.y - self.outcome) * self.sigmoidDerivative(self.outcome), self.weights2.T) * self.sigmoidDerivative(self.HiddenLayer)))

        # Updating weights
        self.weights1 += WeightVal1 * self.learnrate
        self.weights2 += WeightVal2 * self.learnrate     
        
    # Training in data set    
    def dataTraining(self):
        for i in range (self.epoch):
            self.forwardMovement()
            self.backpropogation()
            self.costDataList.append(self.findVals(self.y,self.outcome))

    # Testing in data set        
    def dataPrediction(self,inputData):
        self.input=inputData
        self.forwardMovement()
        return self.outcome
                     

if __name__ == "__main__":

    #Reading CSV file of Training
    data = pd.read_csv("diabetes.csv",header=0)
    Trainingdata=[]
    ClassValue=[]

    #adding data to List for features and its associated output(Whether Diabetic or non-Diabetic)
    for i in range(0,len(data)):
        Trainingdata.append([data.values[i,0],data.values[i,1],data.values[i,2],data.values[i,3],data.values[i,4],data.values[i,5],data.values[i,6],data.values[i,7]])
        if data.values[i,8]==0:
            ClassValue.append([0])
        elif data.values[i,8]==1:
            ClassValue.append([1])
    Trainingdata = findMean(np.array(Trainingdata))        
    #Normalising using sklearn
    scalerData = StandardScaler()
    scalerData.fit(Trainingdata)
    scalerTrainingData=scalerData.transform(Trainingdata)
    

    network = NeuralNetwork(x=scalerTrainingData,y=np.array(ClassValue),learnrate=0.3,epoch=2000)
    #Data Training on Training Data
    network.dataTraining()

    # Reading data for Testing
    testdata = pd.read_csv("diabetes_test.csv",header=0)
    testData=[]

    #Normalising the inputs of test data
    for i in range(0,len(testdata)):
        testData.append([testdata.values[i,0],testdata.values[i,1],testdata.values[i,2],testdata.values[i,3],testdata.values[i,4],testdata.values[i,5],testdata.values[i,6],testdata.values[i,7]])
    
    testData = findMean(np.array(testData))        
    # Data Normalizing  
    xtest=scalerData.transform(testData)
    out=network.dataPrediction(xtest)
    
    ##Labeling the test data based on the output of the Neuralnet
    diabeticType=[]
    print("Output :")
    print("\nPregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  DiabetesPedigreeFunction  Age  Output  DiabetesReport")
    for i in range(0,len(xtest)):
        if(out[i]<0.5):
            #diabeticType.append('Non_Diabetic')
            diabeticType.append('0')
        else:
            #diabeticType.append('Diabetic')
            diabeticType.append('1')
        print(testData[i][0],"    ",testData[i][1],"    ",testData[i][2],"    ",testData[i][3],"    ",testData[i][4],"    ",testData[i][5],"    ",testData[i][6],"    ",testData[i][7],"    ",out[i],"    ",diabeticType[i])


Output :

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  DiabetesPedigreeFunction  Age  Output  DiabetesReport
1.0      108.0      88.0      19.0      163.48192771084337      27.1      0.4      24.0      [0.03559052]      0
6.0      96.0      73.68944099378882      29.629310344827587      163.48192771084337      23.7      0.19      28.0      [0.00052918]      0
1.0      124.0      74.0      36.0      163.48192771084337      27.8      0.1      30.0      [0.0249179]      0
7.0      150.0      78.0      29.0      126.0      35.2      0.6920000000000001      54.0      [0.52458259]      1
4.0      183.0      73.68944099378882      29.629310344827587      163.48192771084337      28.4      0.212      36.0      [0.83177265]      1
1.0      124.0      60.0      32.0      163.48192771084337      35.8      0.514      21.0      [0.45135608]      0
1.0      181.0      78.0      42.0      293.0      40.0      1.258      22.0      [0.50024983]      1
1.0      92.0      62.0      25

2.0      129.0      73.68944099378882      29.629310344827587      163.48192771084337      38.5      0.304      41.0      [0.98966586]      1
4.0      110.0      76.0      20.0      100.0      28.4      0.11800000000000001      27.0      [0.03559017]      0
6.0      80.0      80.0      36.0      163.48192771084337      39.8      0.177      28.0      [0.15828755]      0
10.0      115.0      73.68944099378882      29.629310344827587      163.48192771084337      32.693373493975905      0.261      30.0      [0.09081183]      0
2.0      127.0      46.0      21.0      335.0      34.4      0.17600000000000002      22.0      [0.17166001]      0
9.0      164.0      78.0      29.629310344827587      163.48192771084337      32.8      0.14800000000000002      45.0      [0.51470373]      1
2.0      93.0      64.0      32.0      160.0      38.0      0.674      23.0      [0.04097034]      0
3.0      158.0      64.0      13.0      387.0      31.2      0.295      24.0      [0.61484117]      1
5.0      